In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import librosa
import librosa.display
import soundfile as sf

# Load original audio
audio_path = "/content/Q2 (3).wav"
audio, sr = librosa.load(audio_path)

print("Sample rate:", sr)
print("Duration of the signal in seconds: {:.2f}".format(len(audio)/sr))
ipd.Audio(audio_path)

# Compute the spectrogram of the original audio
original_X = librosa.stft(audio)
magnitude_original = np.abs(original_X)
phase_original = np.angle(original_X)

# Load noise audio
noise_path = "/content/Q2-Noise (2).wav"
noise, sr = librosa.load(noise_path)

print("Sample rate:", sr)
print("Duration of the noise signal in seconds: {:.2f}".format(len(noise)/sr))
ipd.Audio(noise_path)

# Ensure the length of the noise signal matches the original audio
if len(noise) < len(audio):
    noise = np.pad(noise, (0, len(audio) - len(noise)))
else:
    noise = noise[:len(audio)]

# Compute the spectrogram of the noise audio
X_noise = librosa.stft(noise)
magnitude_noise = np.abs(X_noise)
phase_noise = np.angle(X_noise)

# Calculate the mean noise profile
alpha = 2  # Adjust alpha based on experimentation
noise_profile = alpha * np.mean(magnitude_noise, axis=1)

# Tile the mean noise array along the time axis to match dimensions
denoised_audio = magnitude_original - noise_profile[:, np.newaxis]
denoised_audio[denoised_audio < 0] = 0

# Multiply the modified audio by the complex exponential of the phase
denoised_audio_with_phase = denoised_audio * np.exp(1.0j * phase_original)

# Inverse Short-Time Fourier Transform (ISTFT) to reconstruct the audio
denoised_audio_time = librosa.istft(denoised_audio_with_phase)

# Save the denoised audio to a WAV file
sf.write("/content/denoised.wav", denoised_audio_time, sr)

# Load the denoised audio
denoised, sr = librosa.load("/content/denoised.wav")

# Play the denoised audio
ipd.Audio(denoised, rate=sr)
